# Raw Data Processing

This script processes raw data downloaded from original sources. These raw datasets are not available on Github due to their size.

In [2]:
import pandas as pd
import numpy as np

### Fixed Bandwidth

In [16]:
data_19_20 = pd.read_csv(
    '../data/raw_data/large_files/acessos_banda_larga_fixa/Acessos_Banda_Larga_Fixa_2019-2020.csv',
    delimiter=';'
)

data_21 = pd.read_csv(
    '../data/raw_data/large_files/acessos_banda_larga_fixa/Acessos_Banda_Larga_Fixa_2021.csv',
    delimiter=';'
)

df = data_19_20.append(data_21)

df_grouped = df[['Ano', 'Mês', 'Município', 'UF',
                 'Código IBGE Município', 'Acessos']].groupby(['Município', 'UF', 'Código IBGE Município',
                                                               'Ano', 'Mês']).sum()

df_grouped.reset_index(inplace=True)
df_grouped.rename(
    columns={
        'Município': 'city',
        'UF': 'state',
        'Código IBGE Município': 'city_code',
        'Ano': 'year',
        'Mês': 'month',
        'Acessos': 'accesses'},
    inplace=True
)

df_grouped = df_grouped[df_grouped['year'] >= 2020]
df_grouped['date'] = pd.to_datetime(df_grouped['year']*100 + df_grouped['month'], format='%Y%m')
df_grouped.drop(['year', 'month'], axis=1, inplace=True)
df_grouped.reset_index(inplace=True, drop=True)
df_grouped[['city', 'state', 'city_code', 'date', 'accesses']].to_csv('../data/raw_data/fixed_bandwidth.csv')

### Fixed Telephony

In [9]:
data_aut = pd.read_csv(
    '../data/raw_data/large_files/acessos_telefonia_fixa/Acessos_Telefonia_Fixa_Autorizadas.csv',
    delimiter=';',
    low_memory=False
)

data_con = pd.read_csv(
    '../data/raw_data/large_files/acessos_telefonia_fixa/Acessos_Telefonia_Fixa_Concessionarias.csv',
    delimiter=';',
    low_memory=False
)

df = data_aut.append(data_con)

df_grouped = df[['Ano', 'Mês', 'Município', 'UF',
                 'Código IBGE Município', 'Acessos']].groupby(['Município', 'UF', 'Código IBGE Município',
                                                               'Ano', 'Mês']).sum()
df_grouped.reset_index(inplace=True)
df_grouped.rename(
    columns={
        'Município': 'city',
        'UF': 'state',
        'Código IBGE Município': 'city_code',
        'Ano': 'year',
        'Mês': 'month',
        'Acessos': 'accesses'},
    inplace=True
)

df_grouped = df_grouped[df_grouped['year'] >= 2020]
df_grouped['date'] = pd.to_datetime(df_grouped['year']*100 + df_grouped['month'], format='%Y%m')
df_grouped.drop(['year', 'month'], axis=1, inplace=True)
df_grouped.reset_index(inplace=True, drop=True)
df_grouped[['city', 'state', 'city_code', 'date', 'accesses']].to_csv('../data/raw_data/fixed_telephony.csv')

### Mobile Telephony

In [2]:
paths = [
    '../data/raw_data/large_files/acessos_telefonia_movel/Acessos_Telefonia_Movel_202001-202006.csv',
    '../data/raw_data/large_files/acessos_telefonia_movel/Acessos_Telefonia_Movel_202007-202012.csv',
    '../data/raw_data/large_files/acessos_telefonia_movel/Acessos_Telefonia_Movel_2021.csv'
]

df = pd.DataFrame()

for path in paths:
    data = pd.read_csv(path, delimiter=';')
    df_grouped = data[['Ano', 'Mês', 'Município', 'UF', 'Código IBGE Município', 'Tipo de Produto', 
                       'Tecnologia Geração', 'Acessos']].groupby(['Município', 'UF', 'Código IBGE Município', 
                                                                  'Ano', 'Mês', 'Tipo de Produto', 
                                                                  'Tecnologia Geração']).sum()
    del data
    df_grouped.reset_index(inplace=True)
    
    df_grouped.rename(
        columns={
            'Município': 'city',
            'UF': 'state',
            'Código IBGE Município': 'city_code',
            'Ano': 'year',
            'Mês': 'month',
            'Tipo de Produto': 'product_type',
            'Tecnologia Geração': 'technology',
            'Acessos': 'accesses'},
        inplace=True
    )

    df_grouped['product_type'].replace(
        {
            'DADOS': 'data',
            'VOZ': 'voice',
            'VOZ+DADOS': 'voice+data',
            'M2M_ESPECIAL': 'other',
            'M2M_PADRAO': 'other',
            'OCIOSO': 'other',
            'OCIOSO_M2M_ESPECIAL': 'other',
            'M2M': 'other',
            'PONTO_DE_SERVICO': 'other'
        },
        inplace=True
    )

    df = df.append(df_grouped)
    print('finished', path)

df.sort_values(by=['city', 'year', 'month'], inplace=True)
df['date'] = pd.to_datetime(df['year']*100 + df['month'], format='%Y%m')
df.drop(['year', 'month'], axis=1, inplace=True)
df.reset_index(inplace=True, drop=True)

df[['city', 'state', 'city_code', 'date',
    'product_type', 'technology', 'accesses']].to_csv('../data/raw_data/mobile_telephony.csv')

finished acessos_telefonia_movel/Acessos_Telefonia_Movel_202001-202006.csv
finished acessos_telefonia_movel/Acessos_Telefonia_Movel_202007-202012.csv
finished acessos_telefonia_movel/Acessos_Telefonia_Movel_2021.csv


### Cable TV

In [5]:
df = pd.read_csv(
    '../data/raw_data/large_files/acessos_tv_por_assinatura/Acessos_TV_Assinatura.csv',
    delimiter=';',
    low_memory=False
)

df_grouped = df[['Ano', 'Mês', 'Município', 'UF', 
                 'Código IBGE Município', 'Acessos']].groupby(['Município', 'UF', 'Código IBGE Município', 
                                                               'Ano', 'Mês']).sum()
df_grouped.reset_index(inplace=True)
df_grouped.rename(
    columns={
        'Município': 'city',
        'UF': 'state',
        'Código IBGE Município': 'city_code',
        'Ano': 'year',
        'Mês': 'month',
        'Acessos': 'accesses'},
    inplace=True
)

df_grouped = df_grouped[df_grouped['year'] >= 2020]
df_grouped['date'] = pd.to_datetime(df_grouped['year']*100 + df_grouped['month'], format='%Y%m')
df_grouped.drop(['year', 'month'], axis=1, inplace=True)
df_grouped.reset_index(inplace=True, drop=True)
df_grouped[['city', 'state', 'city_code', 'date', 'accesses']].to_csv('../data/raw_data/cable_television.csv')

### Population Info

In [11]:
df = pd.read_excel('../data/raw_data/large_files/Dados municipio.xlsx')

In [12]:
df.rename(
    columns={
        'yr': 'year',
        'codmun7': 'city_code',
        'pop': 'population',
        'municparea': 'area'
    },
    inplace=True
)

df[['city_code', 'year', 'population', 'area']].to_csv('../data/raw_data/population.csv')